In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Input,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os
import cv2
import random
from PIL import Image

In [ ]:
train_path = "../input/binary-class-dataset/Binary Class Dataset/Training"
test_path = "../input/binary-class-dataset/Binary Class Dataset/Testing"
val_path = "../input/binary-class-dataset/Binary Class Dataset/Validation"


In [ ]:
classes = os.listdir("/kaggle/input/binary-class-dataset/Binary Class Dataset/Training")
print(classes)
sample = 5
sample_images = []
sample_name = []

for i in range(sample):
    for img in classes:
        img_name = f'{train_path}/{img}/{random.choice(os.listdir(f"{train_path}/{img}"))}'
        sample_images.append(img_name)
        sample_name.append(img)
        
plt.figure(figsize = (15,15))

for i in range(len(sample_images)):
    plt.subplot(sample,3,i+1)
    plt.imshow(Image.open(sample_images[i]),cmap = 'gray')
    plt.title(sample_name[i])
    plt.axis('off')

In [ ]:
x_train = []

for masktype in os.listdir(train_path):
    
    imgfolder = train_path+"/"+masktype
    
    for img in os.listdir(imgfolder):
        img_path = imgfolder+"/"+img
        img_arr = cv2.imread(img_path)
        img_arr = cv2.resize(img_arr,(224,224))
        x_train.append(img_arr)

x_test = []

for masktype in os.listdir(test_path):
    
    imgfolder = test_path+"/"+masktype
    
    for img in os.listdir(imgfolder):
        img_path = imgfolder+"/"+img
        img_arr = cv2.imread(img_path)
        img_arr = cv2.resize(img_arr,(224,224))
        x_test.append(img_arr)

x_val = []

for masktype in os.listdir(val_path):
    
    imgfolder = val_path+"/"+masktype
    
    for img in os.listdir(imgfolder):
        img_path = imgfolder+"/"+img
        img_arr = cv2.imread(img_path)
        img_arr = cv2.resize(img_arr,(224,224))
        x_val.append(img_arr)


In [ ]:
train_x = np.array(x_train)
test_x = np.array(x_test)
val_x = np.array(x_val)


In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,zoom_range = 0.2,width_shift_range=0.2,height_shift_range=0.2,horizontal_flip=True, fill_mode="nearest") 
test_datagen = ImageDataGenerator(rescale = 1./255,zoom_range=0.2)
val_datagen = ImageDataGenerator(rescale = 1./255,zoom_range=0.2)

In [ ]:
training_set = train_datagen.flow_from_directory(train_path,target_size = (224,224),batch_size = 32,class_mode = 'sparse')
test_set = test_datagen.flow_from_directory(test_path,target_size = (224,224),batch_size = 32,class_mode = 'sparse')
valid_set = test_datagen.flow_from_directory(val_path,target_size = (224,224),batch_size = 32,class_mode = 'sparse')

In [ ]:
train_y = training_set.classes
test_y = test_set.classes
valid_y = valid_set.classes 

In [ ]:
training_set.class_indices

In [ ]:
print(train_y.shape)
print(test_y.shape)
print(valid_y.shape)

In [ ]:
image_size = [224,224]
baseModel = tf.keras.applications.MobileNetV2(input_shape = image_size+[3],weights='imagenet',include_top=False )


In [ ]:
headModel = baseModel.output
headModel = tf.keras.layers.AveragePooling2D(pool_size = (7,7))(headModel)
headModel = tf.keras.layers.Flatten()(headModel)
headModel = tf.keras.layers.Dense(256, activation = 'relu')(headModel)
headModel = tf.keras.layers.Dense(128, activation = 'relu')(headModel)
headModel = Dropout(0.3)(headModel)
headModel = tf.keras.layers.Dense(64, activation = 'relu')(headModel)
headModel = tf.keras.layers.Dense(16, activation = 'relu')(headModel)
headModel = tf.keras.layers.Dense(2, activation = 'softmax')(headModel)

In [ ]:
model = tf.keras.models.Model(inputs = baseModel.input, outputs = headModel)

In [ ]:
for layers in baseModel.layers:
    baseModel.trainable = False

In [ ]:
lr = 0.00001
opt = tf.keras.optimizers.Adam(learning_rate=lr)
model.compile(optimizer=opt, loss = tf.keras.losses.SparseCategoricalCrossentropy(),metrics = ['accuracy'])

In [ ]:
his = model.fit(train_x,train_y,validation_data = (val_x,valid_y),epochs = 50, batch_size = 32, shuffle=True)


In [ ]:
model.evaluate(test_x,test_y,batch_size = 32)

In [ ]:
val = his.history
print(val.keys())

In [ ]:
import matplotlib.pyplot as plt

x=his
plt.figure(figsize=(10,10))
plt.suptitle('Train Acc vs Val Acc For 00 epochs', fontsize=10)

plt.plot(x.history['accuracy'], label='train acc',marker='.')
plt.plot(x.history['val_accuracy'], label='val acc',marker='.')
plt.legend()
plt.savefig("Accuracy_0epochs.png")
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.suptitle('Train Loss vs Val Loss For 00 epochs', fontsize=10)
plt.ylabel('Accuracy', fontsize=16)
plt.plot(x.history['loss'], label='train loss',marker='.')
plt.plot(x.history['val_loss'], label='val loss',marker='.')
plt.legend()
plt.savefig("Loss_00epochs.png")
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import numpy as np

In [ ]:
y_pred = model.predict(test_x)
y_pred = np.argmax(y_pred,axis = 1)


In [ ]:
print(classification_report(y_pred,test_y))

In [ ]:
from sklearn import metrics
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.pyplot as plt

con=metrics.confusion_matrix(y_pred,test_y)
fig, ax = plot_confusion_matrix(conf_mat=con)
plt.show()

In [ ]:
conf = confusion_matrix(y_pred,test_y)
print(conf)

In [ ]:
print("[INFO] saving mask detector model...")

model.save("MobileNetV250.model", save_format="h5")